In [9]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

# Image Processing

This python program was created to, process the images, which will be used to train our CNN. 

The program loads the images from the 'Puctures' directory, then rotates them 90 degrees clockwize, to make them appear up right.


In [10]:
'''
Made with help from:
1. https://www.geeksforgeeks.org/image-processing-in-python/
2. ChatGPT
'''

dir = '../Local_Things/Pictures/'  # Ensure correct path

plot = False

# Image list sorted by modification timestamp
image_list = sorted(
    [img for img in os.listdir(dir) if img.endswith('.jpg')]
)

# Image rotation parameters
angle = -90
scale = 1

# List of processed images
p_images = []

for img_name in image_list:
    img_path = os.path.join(dir, img_name)
    img = cv2.imread(img_path)
    
    if img is None:
        print(f"Skipping {img_name}, could not read.")
        continue

    try:
        # Convert BGR to RGB
        image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # resize image to 150 px by 150 px
        resized_image = cv2.resize(src = image_rgb, 
                          dsize=(150, 150), 
                          interpolation=cv2.INTER_CUBIC)
        
        # Define rotation parameters
        center = (resized_image.shape[1] // 2, resized_image.shape[0] // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, scale)
        
        # Rotate image
        rotated_image = cv2.warpAffine(resized_image, rotation_matrix, (resized_image.shape[1], resized_image.shape[0]))

        # add rotated_image to p_images
        p_images.append(rotated_image)

        # Plot rotated image 
        if plot:
            plt.figure(figsize=(5, 5))  # Create new figure
            plt.imshow(rotated_image)
            plt.title(f'Rotated Image: {img_name}')
            plt.xticks([])  # Remove x-axis ticks
            plt.yticks([])  # Remove y-axis ticks
            plt.show()

        # Free memory
        del img, image_rgb, rotated_image, resized_image
    except Exception as e:
        print(f"Error processing {img_name}: {e}")


# CNN Model Training

We're using a sequencial model from the getting started section of the tensorflow website:
https://www.tensorflow.org/tutorials/quickstart/beginner

In [ ]:
# convert p_images to numpy.array:

# Convert processed images to a NumPy array
X = np.array(p_images, dtype=np.float32) / 255.0  # Normalize

# Dummy labels (replace with actual labels if available)
num_classes = 3  # Adjust based on your dataset
y = np.random.randint(0, num_classes, size=(len(p_images),))

# One-hot encode labels
y = tf.keras.utils.to_categorical(y, num_classes)

# Split into training and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now X_train, y_train, X_test, y_test are ready for TensorFlow


# mnist = tf.keras.datasets.mnist

# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train, x_test = x_train / 255.0, x_test / 255.0

# Define a CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3)  # No softmax here, since we'll use `from_logits=True`
])

print(y_train.shape)

# Compile the model with correct loss function
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

# Ensure x_train and y_train are in the correct format
print(f"x_train shape: {x_train.shape}")  # Should be (num_samples, 150, 150, 3)
print(f"y_train shape: {y_train.shape}")  # Should be (num_samples,)

# Train the model
model.fit(x_train, y_train, epochs=5)

# Evaluate on test set
model.evaluate(x_test, y_test, verbose=2)

# Create a probability model for predictions
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

# Get predictions
predictions = probability_model(x_test[:5])
print(predictions)


(132, 3)
x_train shape: (132, 150, 150, 3)
y_train shape: (132, 3)
Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 184ms/step - accuracy: 0.3703 - loss: 3.0792
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 191ms/step - accuracy: 0.3288 - loss: 1.2722
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 193ms/step - accuracy: 0.3757 - loss: 1.0913
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 183ms/step - accuracy: 0.4134 - loss: 1.0928
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 224ms/step - accuracy: 0.3862 - loss: 1.0910
2/2 - 0s - 173ms/step - accuracy: 0.2121 - loss: 1.1256
tf.Tensor(
[[0.29544908 0.41090515 0.2936458 ]
 [0.28559667 0.42834324 0.28606012]
 [0.312504   0.38429156 0.30320442]
 [0.31165096 0.38531545 0.3030336 ]
 [0.28959304 0.42492247 0.28548443]], shape=(5, 3), dtype=float32)
